# Descente de gradient pour la régression logistique

## Objectifs
Dans ce TP, vous allez :
- mettre à jour la descente de gradient pour la régression logistique.
- explorer la descente de gradient sur un ensemble de données que vous connaissez déjà.

In [ ]:
import copy, math
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from lab_utils_common import  dlc, plot_data, plt_tumor_data, sigmoid, compute_cost_logistic
from plt_quad_logistic import plt_quad_logistic, plt_prob
plt.style.use('./deeplearning.mplstyle')

## Ensemble de données
Commençons par le même ensemble de données à deux caractéristiques utilisé dans le TP de la frontière de décision.

In [ ]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_train = np.array([0, 0, 0, 1, 1, 1])

Comme précédemment, nous utiliserons une fonction externe pour tracer ces données. 
Les points de données avec l'étiquette $y=1$ sont représentés par des croix rouges, tandis que les points de données avec l'étiquette $y=0$ sont représentés par des cercles bleus.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X_train, y_train, ax)

ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

## Descente de gradient logistique

Rappelez-vous que l'algorithme de descente de gradient utilise le calcul du gradient :
$$\begin{align*}
&\text{répéter jusqu'à convergence :} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{pour j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$

Où chaque itération effectue des mises à jour simultanées sur $w_j$ pour tous les $j$, où
$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3} 
\end{align*}$$

* m est le nombre d'exemples d'entraînement dans l'ensemble de données      
* $f_{\mathbf{w},b}(x^{(i)})$ est la prédiction du modèle, tandis que $y^{(i)}$ est la cible
* Pour un modèle de régression logistique  
    $z = \mathbf{w} \cdot \mathbf{x} + b$  
    $f_{\mathbf{w},b}(x) = g(z)$  
    où $g(z)$ est la fonction sigmoïde :  
    $g(z) = \frac{1}{1+e^{-z}}$

### Implémentation de la Descente de Gradient
L'implémentation de l'algorithme de descente de gradient a deux composants :
- La boucle utlisant l'équation (1) ci-dessus. C'est `gradient_descent` et elle vous est généralement fournie dans les TPs optionnels et pratiques.
- Le calcul du gradient actuel, équations (2,3). C'est la fonction `compute_gradient_logistic`.
- On vous demandera de l'implémenter dans le TP pratique de cette semaine.

#### Calcul du Gradient, Description du Code
Implémentez l'équation (2),(3) ci-dessus pour tous les $w_j$ et $b$.
Il existe de nombreuses façons de le faire, mais voici une proposition : 
- initialiser les variables pour accumuler `dj_dw` et `dj_db`
- pour chaque exemple
    - calculer l'erreur pour cet exemple $g(\mathbf{w} \cdot \mathbf{x}^{(i)} + b) - \mathbf{y}^{(i)}$
    - pour chaque valeur d'entrée $x_{j}^{(i)}$ dans cet exemple,
        - multiplier l'erreur par l'entrée $x_{j}^{(i)}$, et ajouter à l'élément correspondant de `dj_dw`. (équation 2 ci-dessus)
    - ajouter l'erreur à `dj_db` (équation 3 ci-dessus)

- diviser `dj_db` et `dj_dw` par le nombre total d'exemples (m)
- notez que $\mathbf{x}^{(i)}$ avec numpy est `X[i,:]` ou `X[i]` et que $x_{j}^{(i)}$ est `X[i,j]`

Veuillez l'implémenter dans la cellule ci-dessous.

In [ ]:
def compute_gradient_logistic(X, y, w, b): 
    """
    Calcule le gradient pour la régression logistique
 
    Args:
      X (ndarray (m,n)): Données, m exemples avec n caractéristiques
      y (ndarray (m,)): valeurs cibles
      w (ndarray (n,)): paramètres du modèle  
      b (scalaire)    : paramètre du modèle
    Returns
      dj_dw (ndarray (n,)): Le gradient du coût par rapport aux paramètres w. 
      dj_db (scalaire)    : Le gradient du coût par rapport au paramètre b. 
    """
        
    return dj_db, dj_dw  

Vérifiez son implémentation avec la celle ci-dessous :

In [ ]:
X_tmp = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_tmp = np.array([0, 0, 0, 1, 1, 1])
w_tmp = np.array([2.,3.])
b_tmp = 1.
dj_db_tmp, dj_dw_tmp = compute_gradient_logistic(X_tmp, y_tmp, w_tmp, b_tmp)
print(f"dj_db: {dj_db_tmp}" )
print(f"dj_dw: {dj_dw_tmp.tolist()}" )

**Résultat attendu**
``` 
dj_db: 0.49861806546328574
dj_dw: [0.498333393278696, 0.49883942983996693]
```

#### Code de la Descente de Gradient
Le code mettant en œuvre l'équation (1) ci-dessus est implémenté juste après. Prenez un moment pour localiser et comparer les fonctions utilisée dans cette fonction  avec les équations ci-dessus.

In [ ]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters): 
    """
    Effectue la descente de gradient en batch
    
    Args:
      X (ndarray (m,n))   : Données, m exemples avec n caractéristiques
      y (ndarray (m,))    : valeurs cibles
      w_in (ndarray (n,)) : Valeurs initiales des paramètres du modèle  
      b_in (scalaire)     : Valeur initiale du paramètre du modèle
      alpha (float)       : Taux d'apprentissage
      num_iters (scalaire): nombre d'itérations pour exécuter la descente de gradient
      
    Returns:
      w (ndarray (n,))    : Valeurs mises à jour des paramètres
      b (scalaire)        : Valeur mise à jour du paramètre 
    """
    # Un tableau pour stocker le coût J et les w à chaque itération principalement pour le graphique plus tard
    J_history = []
    w = copy.deepcopy(w_in)  #éviter de modifier le w global dans la fonction
    b = b_in
    
    for i in range(num_iters):
        # Calcule le gradient et met à jour les paramètres
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)   

        # Met à jour les paramètres en utilisant w, b, alpha et le gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
        # Enregistre le coût J à chaque itération
        if i<100000:      # prévenir l'épuisement des ressources 
            J_history.append( compute_cost_logistic(X, y, w, b) )

        # Imprime le coût à des intervalles de 10 fois ou autant d'itérations si < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Coût {J_history[-1]}   ")
        
    return w, b, J_history         #retourne le w, b final et l'historique de J pour le graphique

Exécutons la descente de gradient sur notre ensemble de données.

In [ ]:
w_tmp  = np.zeros_like(X_train[0])
b_tmp  = 0.
alph = 0.1
iters = 10000

w_out, b_out, _ = gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters) 
print(f"\Nouveaux paramètres: w:{w_out}, b:{b_out}")

#### Graphique du résultat de la descente de gradient : 

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
# affiche la probabilité
plt_prob(ax, w_out, b_out)

# donnée originales 
ax.set_ylabel(r'$x_1$')
ax.set_xlabel(r'$x_0$')   
ax.axis([0, 4, 0, 3.5])
plot_data(X_train,y_train,ax)

# frontière de décision
x0 = -b_out/w_out[0]
x1 = -b_out/w_out[1]
ax.plot([0,x0],[x1,0], c=dlc["dlblue"], lw=1)
plt.show()

Dans le graphique ci-dessus :
 - l'ombrage reflète la probabilité y=1 (résultat avant la frontière de décision)
 - la frontière de décision est la ligne où la probabilité = 0,5

## Un autre ensemble de données
Revenons à un ensemble de données à une variable. Avec seulement deux paramètres, $w$, $b$, il est possible de tracer la fonction de coût en utilisant un graphique de contour pour avoir une meilleure idée de ce que fait la descente de gradient.

In [ ]:
x_train = np.array([0., 1, 2, 3, 4, 5])
y_train = np.array([0,  0, 0, 1, 1, 1])

Comme précédemment, nous utiliserons une fonction externe pour tracer ces données. Les points de données avec l'étiquette $y=1$ sont représentés par des croix rouges, tandis que les points de données avec l'étiquette $y=0$ sont représentés par des cercles bleus.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
plt_tumor_data(x_train, y_train, ax)
plt.show()

Dans le graphique ci-dessous, essayez :
- de changer $w$ et $b$ en cliquant à l'intérieur du graphique de contour en haut à droite.
    - les changements peuvent prendre une ou deux secondes
    - notez la valeur changeante du coût sur le graphique en haut à gauche.
    - notez que le coût est accumulé par une perte sur chaque exemple (lignes verticales pointillées)
- exécutez la descente de gradient en cliquant sur le bouton orange.
    - notez le coût qui diminue régulièrement (le contour et le graphique du coût sont en log(coût)
    - cliquer dans le graphique de contour réinitialisera le modèle pour une nouvelle exécution
- pour réinitialiser le graphique, réexécutez la cellule

In [ ]:
w_range = np.array([-1, 7])
b_range = np.array([1, -14])
quad = plt_quad_logistic( x_train, y_train, w_range, b_range )

## Félicitations !
Vous avez :
- examiné les formules et l'implémentation du calcul du gradient pour la régression logistique
- utilisé ces routines dans
    - l'exploration d'un ensemble de données à une seule variable
    - l'exploration d'un ensemble de données à deux variables